# How might we generate JS/TS classes from ipywidgets?

This is a companion piece to [@jtpio/jupyterlite#whatever](#whatever).

The goal would be to arrive at a stably-versioned build of the `HasTraits` and `ipywidgets` APIs (without lower level `traitlets` internals) built on top of generic comms.

In [ ]:
from traitlets import *
from ipywidgets import *

import jsonschema
import pprint
import warnings
from pathlib import Path
import IPython
import json
import tempfile
import jinja2
import subprocess

## Our work cut out for us

Eventually, we'll open this up for everything.

In [ ]:
def walk_subs(c):
    yield c
    for sub in c.__subclasses__():
        for sub_sub in walk_subs(sub):
            yield sub_sub
    
widget_classes = set(walk_subs(Widget))
IPython.display.Markdown(f"""# {len(widget_classes)} Widget classes""")

In [ ]:
wc = FloatSlider
i = wc()
i.keys, i.get_state()

## We speak schema

In [ ]:
BASE = {
    "type": "object",
    "$schema": "http://json-schema.org/draft-07/schema#",
    # might regret this...
    "additionalProperties": False,
}

### Always be validating

In [ ]:
VALIDATING = True

### Stuff we're not going to handle right now

In [ ]:
IGNORED_TRAITS = [
    "_display_callbacks",
    "_msg_callbacks",
    "_property_lock",
    "comm",
    "layout",
    "log",
    "readout_format",
    "style",
    "handle_color",
    "bar_color",
    "button_color",
    "keys"
]

### How to get make a default (WIP)

In [ ]:
TO_DEFAULT = lambda t_name, i, s: {
    "default": s.get(t_name) if t_name in s else {}
}

### How to get any description

In [ ]:
TO_DESCRIPTION = lambda t_def: {"description": t_def.help}

### How to map traits to schema (WIP)

In [ ]:
TYPE_TO_SCHEMA = {
    str: {"type": "string"},
    set: {"type": "string", "uniqueItems": True},
    int: {"type": "integer"},
    float: {"type": "number"},
    bool: {"type": "boolean"}
}
TRAIT_TO_SCHEMA = {
    Unicode: lambda t_name, t_def, i, s: {"type": "string"},
    Float: lambda t_name, t_def, i, s: {"type": "number"},
    Int: lambda t_name, t_def, i, s: {"type": "integer"},
    Bool: lambda t_name, t_def, i, s: {"type": "boolean"},
    CaselessStrEnum: lambda t_name, t_def, i, s: {"enum": t_def.values},
    List: lambda t_name, t_def, i, s: {
        "type": "array",
        "items": {"description": f"TODO: {t_name} = List({t_def.__dict__})"}
    },
    Tuple: lambda t_name, t_def, i, s: {
        "type": "array",
        "items": {"description": f"TODO: {t_name} = Tyuple({t_def.__dict__})"}
    },
    Set: lambda t_name, t_def, i, s: {
        "type": "array",
        "uniqueItems": True,
        "items": {"description": f"TODO: {t_name} = Set({t_def.__dict__})"}
    },
    Dict: lambda t_name, t_def, i, s: {
        "type": "object",
        "description": f"TODO: {t_name} = Dict({t_def.__dict__})"
    },
    trait_types.TypedTuple: lambda t_name, t_def, i, s: {
        "type": "array",
        "items": {
            **(
                {} if t_def._trait.__class__ not in TRAIT_TO_SCHEMA
                else TRAIT_TO_SCHEMA[t_def._trait.__class__](t_name, t_def, i, s)
            )
        }
    },
    Any: lambda t_name, t_def, i, s: dict()
}
TRAIT_TO_SCHEMA[CUnicode] = TRAIT_TO_SCHEMA[Unicode]
TRAIT_TO_SCHEMA[Color] = TRAIT_TO_SCHEMA[Unicode]
TRAIT_TO_SCHEMA[CInt] = TRAIT_TO_SCHEMA[Int]
TRAIT_TO_SCHEMA[CFloat] = TRAIT_TO_SCHEMA[Float]

### handle one trait

In [ ]:
def one_trait(t_name, t_def, i, s):
    if t_name in IGNORED_TRAITS:
        return
    t_cls = t_def.__class__
    s_def = TRAIT_TO_SCHEMA.get(t_cls)
    
    if s_def:
        non_null = {
            **s_def(t_name, t_def, i, s),
            **TO_DEFAULT(t_name, i, s),
            **TO_DESCRIPTION(t_def)
        }
        
        if getattr(t_def, "allow_none", None):
            return {"oneOf": [non_null, {"type": "null"}]}
        return non_null
    
    warnings.warn(f"{t_name} {t_cls}")

### Make schema

In [ ]:
def clean_state(i=None, wc=None):
    i = i or wc()
    s = {k: v for k, v in i.get_state().items() if k not in IGNORED_TRAITS}
    s = {k: v for k, v in s.items() if not isinstance(v, bytes)}
    return s

In [ ]:
def make_protected_schema(i=None, wc=None, s=None):
    i = i or wc()
    s = s or clean_state(i)
    properties = {
        t_name: one_trait(t_name, t_def, i, s)
        for t_name, t_def in i.traits().items()
        if t_name not in IGNORED_TRAITS and one_trait(t_name, t_def, i, s)
    }
    default_protected = {
        prop: getattr(i, prop) for prop in properties
    }
    default_protected = {
        prop: list(val) if isinstance(val, (set)) else val
        for prop, val in default_protected.items()
    }
    
    return {
        "title": f"{wc.__name__} Protected",
        "description": f"The protected API for {wc.__name__}",
        **BASE,
        "default": default_protected,
        "properties": properties, 
        "required": sorted(default_protected.keys()),
    }

In [ ]:
def make_public_schema(i=None, wc=None, s=None, protected=None):
    i = i or wc()
    s = s or clean_state(i, wc)
    protected = make_protected_schema(i, wc, s)
    in_keys = {
        name: schema for name, schema 
        in protected["properties"].items() 
        if name in i.keys
    }
    
    default_public = {**s}
    return {
        "title": f"{wc.__name__} public",
        "description": f"The public API for {wc.__name__}",
        **BASE,
        "default": default_public,
        "properties": {
            name: protected["properties"][name] for name in in_keys
        },
        "required": sorted(in_keys.keys()),
    }

In [ ]:
def widget_to_json_schemata(i=None, wc=None, s=None):
    """ generate the full public and protected schemata
    """
    i = i or wc()
    s = s or clean_state(i, wc)
    protected = make_protected_schema(i, wc, s)
    public = make_public_schema(i, wc, s, protected)

    return public, protected
for schema in widget_to_json_schemata(wc=wc):
    IPython.display.display(IPython.display.JSON(schema))

### validate

In [ ]:
def validate(i=None, wc=None, schemata=None):
    i = i or wc()
    schemata = schemata or widget_to_json_schemata(i, wc)
    for api in schemata:
        api_default = json.loads(json.dumps(api["default"]))
        errors = [
            err.__dict__
            for err
            in jsonschema.Draft7Validator(api).iter_errors(api_default)
        ]
        if not errors:
            print(wc.__name__, "OK")
        if errors and VALIDATING:
            [
                print(err)
                for err in errors
            ]
            raise ValueError(f"{len(errors)} in {wc.__name__}")
    return schemata
validate(i, wc);

In [ ]:
BASE_JSON_TO_TS = lambda comment: [
    "yarn", "json2ts", "--unreachableDefintions", "--bannerComment", f"""/** 
    {comment} 
    */"""
]

In [ ]:
def make_a_widget_type_description(schemata=None, wc=None):
    schemata = schemata or validate(wc=wc)
    
    final_dts = ""
    
    with tempfile.TemporaryDirectory() as td:
        tdp = Path(td)
        for api in schemata:
            schema_json = tdp / "foo.json"
            schema_json.write_text(json.dumps(api))
            dts = tdp / "foo.d.ts"
            args = [*BASE_JSON_TO_TS(f"@see {wc.__name__}"), "-i", schema_json, "-o", dts]
            subprocess.check_call([*map(str, args)], cwd=Path.cwd().parent)
            final_dts += "\n" + dts.read_text()
            print(len(final_dts))
    return final_dts
make_a_widget_type_description(wc=wc)

In [ ]:
all_schema = {}
for wc in widget_classes:
    try:
        all_schema[wc] = validate(wc=wc)
        IPython.get_ipython().log.info("OK %s", wc.__name__)
    except Exception as err:
        IPython.get_ipython().log.warning(err)
len(all_schema)

In [ ]:
def build_an_any_of_tree(all_schema, wc):
    definitions = {
        f"""I{"Protected" if si else "Public"}{c.__name__}""": schema
        for c, schemata in all_schema.items()
        for si, schema in enumerate(schemata)
    }
    return {
        "title": f"Any {wc.__name__}",
        **BASE,
        "anyOf": [
            {
                "title": "A Public Widget",
                "anyOf": [
                    {"$ref": f"#/definitions/{ref}"}
                     for ref in definitions if ref.startswith("IPublic")
                ]
            },
            {
                "title": "A Protected Widget",
                "anyOf": [
                    {"$ref": f"#/definitions/{ref}"}
                     for ref in definitions if ref.startswith("IProtected")
                ]
            },
        ],
        "definitions": definitions,
    }
any_of_tree = build_an_any_of_tree(all_schema, wc=Widget)

In [ ]:
all_dts = make_a_widget_type_description(schemata=[any_of_tree], wc=Widget)

In [ ]:
any_of_tree["definitions"].keys()

## Write out the types

In [ ]:
proto = Path.cwd().parent / "packages/kernel/src/proto_widgets.ts"
assert proto.exists()
kernel_src = proto.parent
_schema_widgets = kernel_src / "_schema_widgets.d.ts"

In [ ]:
_schema_widgets.write_text(all_dts)

## Write out the JSON

In [ ]:
_schema_json = kernel_src / "_schema_widgets.json"
json_dump = dict(sum([
    [(f"IPublic{wc.__name__}", public), (f"IProtected{wc.__name__}", protected)] 
    for wc, [public, protected] in all_schema.items()
], []))
_schema_json.write_text(json.dumps(
json_dump,
indent=2
))

## More classes

In [ ]:
HEADER = """
/** a bunch of widgets */
import * as PROTO from './_schema_widgets';
import * as SCHEMA from './_schema_widgets.json';
import {_HasTraits, _Widget} from './proto_widgets';
export let ALL = {} as Record<string, any>;
"""

In [ ]:
FOOTER = """
// fin
"""

In [ ]:
TEMPLATE = jinja2.Template("""{{ HEADER }}
{% for ns, wcs in widget_classes|groupby("ns") %}
export namespace {{ ns | join("_") }} {
    {% for item in wcs %}
    {% set wc = item.wc %}
    {% set n = wc.__name__ %}
    {% set scopes = ["Public", "Protected"] %}
    {% if n not in ["Widget", "interactive"] %}
    {% set t = "TAny" + n %}
    /** a type for the traits of {{ n }}*/
    export type {{ t }} = {% for s in scopes %}{% if loop.index0 %} | {% endif %}PROTO.{{ n }}{{ s }}{% endfor %};

    /** a naive {{ n }} 

    {{ wc.__doc__ }}

    @see https://ipywidgets.readthedocs.io/en/7.6.3/examples/Widget%20List.html#{{ wc.__name__ }}
    */
    export class _{{ n }} extends _Widget<{{ t }}> {
      constructor(options: {{ t }}) {
        super({ ..._{{ n }}.defaults(), ...options });
      }

      static defaults(): {{ t }} {
        return {
          ...super.defaults(),
          {%- for s in scopes %}
          ...SCHEMA.I{{ s }}{{ n }}.default,{% endfor %}
        };
      }
    }

    /** the concrete observable {{ n }} */
    export const {{ n }} = _HasTraits._traitMeta<{{ t }}>(_{{ n }}); 
    
    if (!ALL['{{ n }}']) {
        ALL['{{ n }}'] = {{ n }};
    } else {
        console.log('{{ n }} is already hoisted', ALL['{{ n }}']);
    }
    {% endif %}
    // ---
    {% endfor %}
} // end of {{ ns }}
{% endfor %}
{{ FOOTER }}""")

In [ ]:
some_ts = TEMPLATE.render(widget_classes=[
        {"wc": wc, "ns": wc.__module__.split(".")}
        for wc, [public, protected] in all_schema.items()
], HEADER=HEADER, FOOTER=FOOTER)

In [ ]:
IPython.display.Markdown(f"""```ts
{some_ts.split("// ---")[0]}
```""")

In [ ]:
_ts_proto_wrappers = kernel_src / "_proto_wrappers.ts"
_ts_proto_wrappers.write_text(some_ts)

## Prettier

In [ ]:
!cd .. && node ./node_modules/.bin/prettier --write packages/kernel/src/*